## Dataset EDA Notebook

In [4]:
import pandas as pd

from thefuzz import process


In [5]:
file_path = "../../data/Food_Inspections_20250216.csv"
df = pd.read_csv(file_path, header=[0])
df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,1106427,BLOOMING BUD DAYCARE,BLOOMING BUD DAYCARE,2215789.0,Daycare Combo 1586,Risk 1 (High),5715 N LINCOLN AVE,CHICAGO,IL,60659.0,03/07/2013,License,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.985390,-87.698734,"(41.98538950526786, -87.69873407149943)"
1,2608378,Babas Halal,Babas Halal,2684170.0,Restaurant,Risk 1 (High),7901 S DAMEN AVE,CHICAGO,IL,60620.0,12/03/2024,Complaint,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.750189,-87.672986,"(41.750189342293375, -87.67298583977204)"
2,1106406,FIRST ZABIHA MEAT BAZAAR,FIRST ZABIHA MEAT BAZAAR,2232559.0,Grocery Store,Risk 2 (Medium),2907 W DEVON AVE,CHICAGO,IL,60659.0,02/20/2013,License,Fail,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D...",41.997401,-87.702385,"(41.99740137039031, -87.70238538227812)"
3,2609909,HAPPY MARKET,HAPPY MARKET,2912802.0,Grocery Store,Risk 2 (Medium),2334 S WENTWORTH AVE,CHICAGO,IL,60616.0,01/02/2025,Canvass,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.849954,-87.632094,"(41.84995400192252, -87.63209419559098)"
4,2609927,SAT KAIVAL FOOD INC/SUBWAY,SAT KAIVAL FOOD INC/SUBWAY,2728400.0,Restaurant,Risk 1 (High),1916 S STATE ST,CHICAGO,IL,60616.0,01/02/2025,Canvass,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.856053,-87.627311,"(41.85605269621059, -87.62731125804903)"


In [6]:
# renaming column names to snake_case
COLUMN_NAMES = [
    'inspection_id',
    'dba_name',
    'aka_name',
    'license_',
    'facility_type',
    'risk',
    'address',
    'city',
    'state',
    'zip',
    'inspection_date',
    'inspection_type',
    'results',
    'violations',
    'latitude',
    'longitude',
    'location'
]

df.columns = COLUMN_NAMES

### Summary Statistics

In [7]:
def calculate_summary_stats(df, column_name):


    if column_name in df.columns:
        num_records = len(df)
        num_na = df[column_name].isna().sum()
        num_unique = df[column_name].nunique()

        print(f"Summary stats for '{column_name}'")
        print(f"Missing:         {num_na} ({round(num_na/num_records,1)}%)")
        print(f"Distinct:        {num_unique} ({round(num_unique/num_records,1)}%)")
        print("\n")
    
    else:
        print(f"{column_name} is not in dataframe")

In [8]:
for column in COLUMN_NAMES:
    calculate_summary_stats(df, column)

Summary stats for 'inspection_id'
Missing:         0 (0.0%)
Distinct:        287237 (1.0%)


Summary stats for 'dba_name'
Missing:         0 (0.0%)
Distinct:        33304 (0.1%)


Summary stats for 'aka_name'
Missing:         2426 (0.0%)
Distinct:        31703 (0.1%)


Summary stats for 'license_'
Missing:         18 (0.0%)
Distinct:        46274 (0.2%)


Summary stats for 'facility_type'
Missing:         5197 (0.0%)
Distinct:        519 (0.0%)


Summary stats for 'risk'
Missing:         88 (0.0%)
Distinct:        4 (0.0%)


Summary stats for 'address'
Missing:         0 (0.0%)
Distinct:        30905 (0.1%)


Summary stats for 'city'
Missing:         164 (0.0%)
Distinct:        86 (0.0%)


Summary stats for 'state'
Missing:         58 (0.0%)
Distinct:        6 (0.0%)


Summary stats for 'zip'
Missing:         39 (0.0%)
Distinct:        127 (0.0%)


Summary stats for 'inspection_date'
Missing:         0 (0.0%)
Distinct:        3813 (0.0%)


Summary stats for 'inspection_type'
Missing:  

### Inspection ID

Check if inspection_id is a UUID

In [9]:
if df['inspection_id'].nunique() == len(df):
    print("Number of records matches the total distinct inspection IDs")

Number of records matches the total distinct inspection IDs


inspection_id is indeed UUID, this field is unnecessary for FDs

### DBA Name and AKA Name

The description for DBA is Doing Business As, and for AKA its Also Known As. Since both are essentially variations of the business name, only one of them is necessary

Since there are missing values for 'aka_name', lets drop this and use dba_name instead

### Location Details

Address, City, State, Zip

In [10]:
df['state'].nunique()

6

Since the missing values for city, state and zip are less than 0.1% of the entire dataset, they can be dropped

zip is currently a float, it should be an integer

### License

In [11]:
df['license_'].nunique()

46274

In [12]:
df.groupby(by='license_')['license_'].count().reset_index(name="count")

,license_,count
0,0.0,743
1,1.0,1
2,2.0,17
3,9.0,15
4,40.0,21
...,...,...
46269,4080337.0,1
46270,4080354.0,1
46271,5852857.0,1
46272,8700606.0,3


In [13]:
import pandas as pd
from thefuzz import process, fuzz
from collections import defaultdict

def fuzzy_normalize_column(df, column_name, threshold=80):
    """
    Normalize text values in a DataFrame column using fuzzy matching.
    
    Args:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Column name to normalize.
    - threshold (int): Similarity threshold for fuzzy matching (default is 80).
    
    Returns:
    - pd.DataFrame: DataFrame with a new normalized column.
    """
    df[column_name] = df[column_name].astype(str).fillna('')  # Convert to string

    unique_values = list(set(df[column_name].str.lower()))  # Unique values in lowercase

    # Reference mapping for normalization
    reference_mapping = {}
    groups = defaultdict(list)  # To store word clusters

    for value in unique_values:
        # Check if it's already in a group
        if value in reference_mapping:
            continue
        
        # Find similar words
        matches = process.extract(value, unique_values, limit=10, scorer=fuzz.ratio)
        matches = [(match, score) for match, score in matches if score >= threshold]
        
        if matches:
            best_match = max(matches, key=lambda x: x[1])[0]  # Pick the best-scoring match
        else:
            best_match = value  # Keep original if no good match found

        # Assign all similar words to the best match
        for match, score in matches:
            reference_mapping[match] = best_match
            groups[best_match].append(match)

    # Apply normalization mapping
    df[f'{column_name}_normalised'] = df[column_name].str.lower().map(reference_mapping)
    
    return df


In [14]:
df.groupby(by='risk')['risk'].count().reset_index(name="count")

,risk,count
0,All,68
1,Risk 1 (High),211761
2,Risk 2 (Medium),52347
3,Risk 3 (Low),22973


### Facility Type

### Geographic Coordinates

Latitude, Longitude and Location

Location appears to be a tuple of latitude and longtitude values. Since most geographic systems uses lat and lon as separate values, lets drop Location.

There are 978 records with missing coordinates, which is less than 0.1% of the dataset. These records can be dropped. 

## Performing preprocessing

- Rename columns
- Drop the following columns - inspection_id, aka_name, location
- Drop missing records for city, state, zip, latitude, longitude
- Fix data type
  - license: float -> int
  - zip: float -> int

In [15]:
# renaming column names to snake_case
COLUMN_NAMES = [
    'inspection_id',
    'dba_name',
    'aka_name',
    'license_',
    'facility_type',
    'risk',
    'address',
    'city',
    'state',
    'zip',
    'inspection_date',
    'inspection_type',
    'results',
    'violations',
    'latitude',
    'longitude',
    'location'
]

df.columns = COLUMN_NAMES

# drop columns
df.drop(['inspection_id', 'aka_name', 'location'], axis=1, inplace=True)

# drop missing values
df.dropna(subset=['city', 'state', 'zip', 'latitude', 'longitude'], inplace=True)

# fix data type
df = df.astype({'zip':'Int64', 'license_':'Int64'})

# consolidate redundant values using fuzzy matching
df = fuzzy_normalize_column(df, 'inspection_type', threshold=70)


In [16]:
df

,dba_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,inspection_type_normalised
0,BLOOMING BUD DAYCARE,2215789,Daycare Combo 1586,Risk 1 (High),5715 N LINCOLN AVE,CHICAGO,IL,60659,03/07/2013,License,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.985390,-87.698734,license
1,Babas Halal,2684170,Restaurant,Risk 1 (High),7901 S DAMEN AVE,CHICAGO,IL,60620,12/03/2024,Complaint,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.750189,-87.672986,covid complaint
2,FIRST ZABIHA MEAT BAZAAR,2232559,Grocery Store,Risk 2 (Medium),2907 W DEVON AVE,CHICAGO,IL,60659,02/20/2013,License,Fail,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D...",41.997401,-87.702385,license
3,HAPPY MARKET,2912802,Grocery Store,Risk 2 (Medium),2334 S WENTWORTH AVE,CHICAGO,IL,60616,01/02/2025,Canvass,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.849954,-87.632094,canvas
4,SAT KAIVAL FOOD INC/SUBWAY,2728400,Restaurant,Risk 1 (High),1916 S STATE ST,CHICAGO,IL,60616,01/02/2025,Canvass,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.856053,-87.627311,canvas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287232,DAPPER'S EAST RESTAURANT,12540,Restaurant,Risk 1 (High),2901 W ADDISON ST,CHICAGO,IL,60618,01/23/2025,Complaint Re-Inspection,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.946553,-87.700536,non-inspection
287233,SHARPIE'S,3002203,Restaurant,Risk 3 (Low),1721 N ELSTON AVE,CHICAGO,IL,60642,01/22/2025,License,Not Ready,NaN,41.913729,-87.664223,license
287234,ACE SUSHI @POTASH MARKET,2808766,Restaurant,Risk 1 (High),875 N STATE ST,CHICAGO,IL,60610,01/13/2025,Canvass,Pass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",41.898878,-87.628111,canvas
287235,"Kitchen Chicago, LLC",2119905,Shared Kitchen,Risk 1 (High),324 N LEAVITT ST,CHICAGO,IL,60612,01/22/2025,Canvass,Pass,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...,41.887434,-87.681849,canvas


In [35]:
violation_test = df.violations.iloc[0]
violation_test

'32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: PROVIDE A DRAIN BOARD AT THREE COMPARTMENT SINK. | 32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: OBSERVED RUSTY GREASE TRAP UNDER THE THREE COMPARTMENT SINK, INSTRUCTED TO PAINT AND MAINTAIN.  PROVIDE SPLASH GUARD BETWEEN THE  EXPOSED HAND SINK ON BOTH SIDE AND PREP COUNTER | 34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: INSTRUCTED TO SWEEP FLOOR INSIDE THE FURNACE AREA.  OBSERVED AT SECOND FLOOR UNUSED CLASSROOMS WITH ITEMS STORED ON FLOOR, INSTRUCTED TO ELEVATE 6" OFF THE FLOOR,AND ORGANIZE AREA.  RESEAL BASEBOARD UNDER THE THREE COMPARTMENT SINK IN KITCHEN. | 35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: OBSERVED DEAD BULB IN THE KITCHEN, INSTRUCTED TO REPLACE AND PROVIDE LIGHT SH

In [28]:
split_list

['32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: PROVIDE A DRAIN BOARD AT THREE COMPARTMENT SINK. ',
 ' 32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: OBSERVED RUSTY GREASE TRAP UNDER THE THREE COMPARTMENT SINK, INSTRUCTED TO PAINT AND MAINTAIN.  PROVIDE SPLASH GUARD BETWEEN THE  EXPOSED HAND SINK ON BOTH SIDE AND PREP COUNTER ',
 ' 34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: INSTRUCTED TO SWEEP FLOOR INSIDE THE FURNACE AREA.  OBSERVED AT SECOND FLOOR UNUSED CLASSROOMS WITH ITEMS STORED ON FLOOR, INSTRUCTED TO ELEVATE 6" OFF THE FLOOR,AND ORGANIZE AREA.  RESEAL BASEBOARD UNDER THE THREE COMPARTMENT SINK IN KITCHEN. ',
 ' 35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: OBSERVED DEAD BULB IN THE KITCHEN, INSTRUCTED TO REPLACE AND PRO

In [41]:
df.inspection_type

0                         License
1                       Complaint
2                         License
3                         Canvass
4                         Canvass
                   ...           
287232    Complaint Re-Inspection
287233                    License
287234                    Canvass
287235                    Canvass
287236                    Canvass
Name: inspection_type, Length: 286054, dtype: object

In [51]:
def extract_violation_codes(x):

    split_list = str(x).split("|")

    return set([int(violation.strip().split(" ")[0][:-1]) for violation in split_list])

In [52]:
df['violation_code_set'] = df['violations'].apply(lambda x: extract_violation_codes(x))

ValueError: invalid literal for int() with base 10: 'na'

In [50]:
violation_test = df.violations.iloc[0]
split_list = violation_test.split("|")


set([int(violation.strip().split(" ")[0][:-1]) for violation in split_list])


{32, 34, 35}

In [17]:
for column in COLUMN_NAMES:
    calculate_summary_stats(df, column)

inspection_id is not in dataframe
Summary stats for 'dba_name'
Missing:         0 (0.0%)
Distinct:        33107 (0.1%)


aka_name is not in dataframe
Summary stats for 'license_'
Missing:         18 (0.0%)
Distinct:        45990 (0.2%)


Summary stats for 'facility_type'
Missing:         5161 (0.0%)
Distinct:        513 (0.0%)


Summary stats for 'risk'
Missing:         82 (0.0%)
Distinct:        4 (0.0%)


Summary stats for 'address'
Missing:         0 (0.0%)
Distinct:        30636 (0.1%)


Summary stats for 'city'
Missing:         0 (0.0%)
Distinct:        23 (0.0%)


Summary stats for 'state'
Missing:         0 (0.0%)
Distinct:        1 (0.0%)


Summary stats for 'zip'
Missing:         0 (0.0%)
Distinct:        66 (0.0%)


Summary stats for 'inspection_date'
Missing:         0 (0.0%)
Distinct:        3812 (0.0%)


Summary stats for 'inspection_type'
Missing:         0 (0.0%)
Distinct:        110 (0.0%)


Summary stats for 'results'
Missing:         0 (0.0%)
Distinct:        7 (0.0%)

## Summary Findings

- Removal of rows with null values led to the attribute state having only 1 unique value - this implies that records from other states have faulty data

In [2]:
import pandas as pd

In [3]:
file_path = "../../data/Food_Inspections_Violations_Expanded_with_cleandata.csv"
df = pd.read_csv(file_path, header=[0])
df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Location,Facility Type Cleaned,City Cleaned,Inspection Type Cleaned,raw_violation,violation_number,violation_text,violation_comment,parse_error,error_reason
0,1106427,BLOOMING BUD DAYCARE,BLOOMING BUD DAYCARE,2215789.0,DAYCARE COMBO 1586,Risk 1 (High),5715 N LINCOLN AVE,CHICAGO,IL,60659.0,...,"(41.98538950526786, -87.69873407149943)",DAYCARE COMBO 1586,CHICAGO,LICENSE,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,32.0,FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DE...,PROVIDE A DRAIN BOARD AT THREE COMPARTMENT SINK.,False,NaN
1,1106427,BLOOMING BUD DAYCARE,BLOOMING BUD DAYCARE,2215789.0,DAYCARE COMBO 1586,Risk 1 (High),5715 N LINCOLN AVE,CHICAGO,IL,60659.0,...,"(41.98538950526786, -87.69873407149943)",DAYCARE COMBO 1586,CHICAGO,LICENSE,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,32.0,FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DE...,OBSERVED RUSTY GREASE TRAP UNDER THE THREE COM...,False,NaN
2,1106427,BLOOMING BUD DAYCARE,BLOOMING BUD DAYCARE,2215789.0,DAYCARE COMBO 1586,Risk 1 (High),5715 N LINCOLN AVE,CHICAGO,IL,60659.0,...,"(41.98538950526786, -87.69873407149943)",DAYCARE COMBO 1586,CHICAGO,LICENSE,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",34.0,"FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD RE...",INSTRUCTED TO SWEEP FLOOR INSIDE THE FURNACE A...,False,NaN
3,1106427,BLOOMING BUD DAYCARE,BLOOMING BUD DAYCARE,2215789.0,DAYCARE COMBO 1586,Risk 1 (High),5715 N LINCOLN AVE,CHICAGO,IL,60659.0,...,"(41.98538950526786, -87.69873407149943)",DAYCARE COMBO 1586,CHICAGO,LICENSE,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",35.0,"WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTE...","OBSERVED DEAD BULB IN THE KITCHEN, INSTRUCTED ...",False,NaN
4,2608378,Babas Halal,Babas Halal,2684170.0,RESTAURANT,Risk 1 (High),7901 S DAMEN AVE,CHICAGO,IL,60620.0,...,"(41.750189342293375, -87.67298583977204)",RESTAURANT,CHICAGO,COMPLAINT,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",1.0,"PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLED...",PIC DOESN'T HAVE A CERTIFIED FOOD MANAGERS CER...,False,NaN


In [4]:
col_names = ["_".join(col.lower().split(' ')) for col in df.columns]
df.columns = col_names

In [71]:
df.columns

Index(['inspection_id', 'dba_name', 'aka_name', 'license_#', 'facility_type',
       'risk', 'address', 'city', 'state', 'zip', 'inspection_date',
       'inspection_type', 'results', 'latitude', 'longitude', 'location',
       'facility_type_cleaned', 'city_cleaned', 'inspection_type_cleaned',
       'raw_violation', 'violation_number', 'violation_text',
       'violation_comment', 'parse_error', 'error_reason'],
      dtype='object')

In [5]:
new_df = df.drop(
    ['inspection_id','aka_name','facility_type','city','inspection_type','raw_violation','violation_text','location','violation_comment','error_reason','parse_error','state'],axis=1
)

In [10]:
# drop missing values
new_df.dropna(subset=['city_cleaned', 'zip', 'latitude', 'longitude'], inplace=True)

# fix data type
new_df = new_df.astype({'zip':'Int64', 'license_#':'Int64', 'violation_number':'Int64'})

In [15]:
new_df

,dba_name,license_#,risk,address,zip,inspection_date,results,latitude,longitude,facility_type_cleaned,city_cleaned,inspection_type_cleaned,violation_number
0,BLOOMING BUD DAYCARE,2215789,Risk 1 (High),5715 N LINCOLN AVE,60659,3/7/2013,Pass,41.985390,-87.698734,DAYCARE COMBO 1586,CHICAGO,LICENSE,32
1,BLOOMING BUD DAYCARE,2215789,Risk 1 (High),5715 N LINCOLN AVE,60659,3/7/2013,Pass,41.985390,-87.698734,DAYCARE COMBO 1586,CHICAGO,LICENSE,32
2,BLOOMING BUD DAYCARE,2215789,Risk 1 (High),5715 N LINCOLN AVE,60659,3/7/2013,Pass,41.985390,-87.698734,DAYCARE COMBO 1586,CHICAGO,LICENSE,34
3,BLOOMING BUD DAYCARE,2215789,Risk 1 (High),5715 N LINCOLN AVE,60659,3/7/2013,Pass,41.985390,-87.698734,DAYCARE COMBO 1586,CHICAGO,LICENSE,35
4,Babas Halal,2684170,Risk 1 (High),7901 S DAMEN AVE,60620,12/3/2024,Fail,41.750189,-87.672986,RESTAURANT,CHICAGO,COMPLAINT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
939275,ACE SUSHI @POTASH MARKET,2808766,Risk 1 (High),875 N STATE ST,60610,1/13/2025,Pass,41.898878,-87.628111,RESTAURANT,CHICAGO,CANVASS,55
939276,ACE SUSHI @POTASH MARKET,2808766,Risk 1 (High),875 N STATE ST,60610,1/13/2025,Pass,41.898878,-87.628111,RESTAURANT,CHICAGO,CANVASS,58
939277,"Kitchen Chicago, LLC",2119905,Risk 1 (High),324 N LEAVITT ST,60612,1/22/2025,Pass,41.887434,-87.681849,SHARED KITCHEN,CHICAGO,CANVASS,49
939278,"Kitchen Chicago, LLC",2119905,Risk 1 (High),324 N LEAVITT ST,60612,1/22/2025,Pass,41.887434,-87.681849,SHARED KITCHEN,CHICAGO,CANVASS,55


In [12]:
new_df.to_parquet("../../data/Food_Inspections_Violations_Expanded_with_cleandata.parquet")

In [19]:
from collections import defaultdict
from thefuzz import process, fuzz

In [14]:

def fuzzy_normalize_column(df, column_name, threshold=80):
    """
    Normalize text values in a DataFrame column using fuzzy matching.
    
    Args:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Column name to normalize.
    - threshold (int): Similarity threshold for fuzzy matching (default is 80).
    
    Returns:
    - pd.DataFrame: DataFrame with a new normalized column.
    """
    df[column_name] = df[column_name].astype(str).fillna('')  # Convert to string

    unique_values = list(set(df[column_name].str.lower()))  # Unique values in lowercase

    # Reference mapping for normalization
    reference_mapping = {}
    groups = defaultdict(list)  # To store word clusters

    for value in unique_values:
        # Check if it's already in a group
        if value in reference_mapping:
            continue
        
        # Find similar words
        matches = process.extract(value, unique_values, limit=10, scorer=fuzz.ratio)
        matches = [(match, score) for match, score in matches if score >= threshold]
        
        if matches:
            best_match = max(matches, key=lambda x: x[1])[0]  # Pick the best-scoring match
        else:
            best_match = value  # Keep original if no good match found

        # Assign all similar words to the best match
        for match, score in matches:
            reference_mapping[match] = best_match
            groups[best_match].append(match)

    # Apply normalization mapping
    df[f'{column_name}'] = df[column_name].str.lower().map(reference_mapping)
    
    return df

In [23]:
new_df1 = fuzzy_normalize_column(new_df, 'address')

In [24]:
new_df.address.nunique()

6778

In [25]:
new_df1.address.nunique()

6778